In [1]:
%load_ext lab_black

In [2]:
import os
import pytz
import pathlib
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
import glob

this_dir = pathlib.Path(os.path.abspath(""))
data_dir = this_dir / "data"

In [3]:
r = requests.get("http://sierracounty.ca.gov/628/Sierra-County-Case-Map")

In [4]:
soup = BeautifulSoup(r.text)

In [5]:
content = soup.find("div", id="bodyWrapper")

In [6]:
table = soup.find("table")

In [7]:
tbody = table.tbody

In [8]:
row_list = tbody.find_all("tr")[1:]

In [9]:
dict_list = []

In [10]:
def safetxt(element):
    v = element.text.strip()
    v = v.replace("\u200b", "")
    return v

In [11]:
def safenumber(element):
    v = safetxt(element)
    v = v.replace(",", "")
    v = v.replace(" ", "")
    return v

In [12]:
for row in row_list:
    cell_list = row.find_all("td")
    d = dict(
        county="Sierra",
        area=safetxt(cell_list[0]),
        confirmed_cases=safenumber(cell_list[1]),
    )
    dict_list.append(d)

IndexError: list index out of range

In [13]:
df = pd.DataFrame(dict_list)

In [14]:
rows_to_remove = ["Recovered Cases", "Currently Hospitalized", "Deaths"]

In [15]:
df_filtered = df[~df.area.isin(rows_to_remove)].copy()

In [17]:
df_filtered.loc[
    df_filtered.area == "Located on the East Side of the County", "area"
] = "East"
df_filtered.loc[
    df_filtered.area == "Located on the West Side of the County", "area"
] = "West"

Get date

In [18]:
raw_date_string = tbody.find("td")["data-th"]

In [19]:
latest_date = pd.to_datetime(raw_date_string.replace("Total Cases as of ", "")).date()

Set the date

In [20]:
df_filtered["county_date"] = latest_date

Mark the current date

In [25]:
tz = pytz.timezone("America/Los_Angeles")

In [26]:
today = datetime.now(tz).date()

In [27]:
slug = "sierra"

## Vet

In [28]:
try:
    assert not len(df_filtered) > 2
except AssertionError:
    raise AssertionError("Sierra's area scraper has extra rows")

In [29]:
try:
    assert not len(df_filtered) < 2
except AssertionError:
    raise AssertionError("Sierra's area scraper is missing rows")

## Export

In [30]:
df_filtered.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [31]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [32]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [33]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [34]:
df.to_csv(os.path.join(data_dir / slug / "timeseries.csv"), index=False)